# **CHALLENGE TELECOM X - 2**

In [73]:
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.graphics.gofplots import qqplot
import numpy as np
import scipy as sp
from scipy import stats as st

from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

import statsmodels.api as sm
from statsmodels.formula.api import ols
import statsmodels as stm

import seaborn as sns
from warnings import simplefilter


In [74]:
df = pd.read_csv('dados_normalizados.csv')

In [75]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7267 entries, 0 to 7266
Data columns (total 24 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7267 non-null   object 
 1   Churn             7267 non-null   bool   
 2   gender            7267 non-null   object 
 3   SeniorCitizen     7267 non-null   bool   
 4   Partner           7267 non-null   bool   
 5   Dependents        7267 non-null   bool   
 6   tenure            7267 non-null   int64  
 7   PhoneService      7267 non-null   bool   
 8   MultipleLines     7267 non-null   bool   
 9   InternetService   7267 non-null   object 
 10  OnlineSecurity    7267 non-null   bool   
 11  OnlineBackup      7267 non-null   bool   
 12  DeviceProtection  7267 non-null   bool   
 13  TechSupport       7267 non-null   bool   
 14  StreamingTV       7267 non-null   bool   
 15  StreamingMovies   7267 non-null   bool   
 16  Contract          7267 non-null   object 


In [76]:
# Dicionário de tradução dos nomes das colunas
colunas_traduzidas = {
    'customerID': 'ID_Cliente',
    'Churn': 'Cancelou',
    'gender': 'Genero',
    'SeniorCitizen': 'Idoso',
    'Partner': 'Possui_Conjuge',
    'Dependents': 'Possui_Dependentes',
    'tenure': 'Meses_de_Contrato',
    'PhoneService': 'Servico_Telefone',
    'MultipleLines': 'Multiplas_Linhas',
    'InternetService': 'Tipo_Internet',
    'OnlineSecurity': 'Seguranca_Online',
    'OnlineBackup': 'Backup_Online',
    'DeviceProtection': 'Protecao_Dispositivo',
    'TechSupport': 'Suporte_Tecnico',
    'StreamingTV': 'Streaming_TV',
    'StreamingMovies': 'Streaming_Filmes',
    'Contract': 'Tipo_Contrato',
    'PaperlessBilling': 'Fatura_Papel',
    'PaymentMethod': 'Metodo_Pagamento',
    'Charges.Monthly': 'Valor_Mensal',
    # As três últimas colunas parecem já estar em português, mas se quiser padronizar:
    'Qtd.Servicos': 'Qtd_Servicos',
    'total_servicos': 'Total_Servicos'
}

# Renomear as colunas
df.rename(columns=colunas_traduzidas, inplace=True)


In [77]:
df.head()

,ID_Cliente,Cancelou,Genero,Idoso,Possui_Conjuge,Possui_Dependentes,Meses_de_Contrato,Servico_Telefone,Multiplas_Linhas,Tipo_Internet,...,Streaming_TV,Streaming_Filmes,Tipo_Contrato,Fatura_Papel,Metodo_Pagamento,Valor_Mensal,Charges.Total,daily.account,Qtd_Servicos,Total_Servicos
0,0002-ORFBO,False,Female,False,True,True,9,True,False,DSL,...,True,False,One year,True,Mailed check,65.6,593.30,2.197407,4,4
1,0003-MKNFE,False,Male,False,False,False,9,True,True,DSL,...,False,True,Month-to-month,False,Mailed check,59.9,542.40,2.008889,3,3
2,0004-TLHLJ,True,Male,False,False,False,4,True,False,Fiber optic,...,False,False,Month-to-month,True,Electronic check,73.9,280.85,2.340417,2,2
3,0011-IGKFF,True,Male,True,True,False,13,True,False,Fiber optic,...,True,True,Month-to-month,True,Electronic check,98.0,1237.85,3.173974,5,5
4,0013-EXCHZ,True,Female,True,True,False,3,True,False,Fiber optic,...,True,False,Month-to-month,True,Mailed check,83.9,267.40,2.971111,3,3


In [78]:
df.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7267 entries, 0 to 7266
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   ID_Cliente            7267 non-null   object 
 1   Cancelou              7267 non-null   bool   
 2   Genero                7267 non-null   object 
 3   Idoso                 7267 non-null   bool   
 4   Possui_Conjuge        7267 non-null   bool   
 5   Possui_Dependentes    7267 non-null   bool   
 6   Meses_de_Contrato     7267 non-null   int64  
 7   Servico_Telefone      7267 non-null   bool   
 8   Multiplas_Linhas      7267 non-null   bool   
 9   Tipo_Internet         7267 non-null   object 
 10  Seguranca_Online      7267 non-null   bool   
 11  Backup_Online         7267 non-null   bool   
 12  Protecao_Dispositivo  7267 non-null   bool   
 13  Suporte_Tecnico       7267 non-null   bool   
 14  Streaming_TV          7267 non-null   bool   
 15  Streaming_Filmes     

In [79]:
from sklearn.preprocessing import MinMaxScaler

# Lista com as colunas que você quer normalizar
colunas_para_normalizar = ['Meses_de_Contrato', 'Valor_Mensal', 'Qtd_Servicos', 'Total_Servicos']

# Cria o scaler
scaler = MinMaxScaler()

# Normaliza as colunas selecionadas
df[colunas_para_normalizar] = scaler.fit_transform(df[colunas_para_normalizar])


In [80]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7267 entries, 0 to 7266
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   ID_Cliente            7267 non-null   object 
 1   Cancelou              7267 non-null   bool   
 2   Genero                7267 non-null   object 
 3   Idoso                 7267 non-null   bool   
 4   Possui_Conjuge        7267 non-null   bool   
 5   Possui_Dependentes    7267 non-null   bool   
 6   Meses_de_Contrato     7267 non-null   float64
 7   Servico_Telefone      7267 non-null   bool   
 8   Multiplas_Linhas      7267 non-null   bool   
 9   Tipo_Internet         7267 non-null   object 
 10  Seguranca_Online      7267 non-null   bool   
 11  Backup_Online         7267 non-null   bool   
 12  Protecao_Dispositivo  7267 non-null   bool   
 13  Suporte_Tecnico       7267 non-null   bool   
 14  Streaming_TV          7267 non-null   bool   
 15  Streaming_Filmes     

In [81]:
from sklearn.preprocessing import StandardScaler

# Lista com as colunas a serem transformadas
colunas_para_normalizar = ['Meses_de_Contrato', 'Valor_Mensal', 'Qtd_Servicos', 'Total_Servicos']

# Inicializa o scaler
scaler = StandardScaler()

# Aplica o z-score apenas nas colunas desejadas
df[colunas_para_normalizar] = scaler.fit_transform(df[colunas_para_normalizar])


In [82]:
# Exemplo alternativo com valores em inglês
df['Genero'] = df['Genero'].map({'Male': True, 'Female': False})

In [83]:
Y = df['Cancelou']
x = df.drop(columns=['Cancelou','ID_Cliente'])

In [84]:
x


,Genero,Idoso,Possui_Conjuge,Possui_Dependentes,Meses_de_Contrato,Servico_Telefone,Multiplas_Linhas,Tipo_Internet,Seguranca_Online,Backup_Online,...,Streaming_TV,Streaming_Filmes,Tipo_Contrato,Fatura_Papel,Metodo_Pagamento,Valor_Mensal,Charges.Total,daily.account,Qtd_Servicos,Total_Servicos
0,False,False,True,True,-0.950200,True,False,DSL,False,True,...,True,False,One year,True,Mailed check,0.029206,593.30,2.197407,0.310898,0.310898
1,True,False,False,False,-0.950200,True,True,DSL,False,False,...,False,True,Month-to-month,False,Mailed check,-0.159990,542.40,2.008889,-0.173930,-0.173930
2,True,False,False,False,-1.153700,True,False,Fiber optic,False,False,...,False,False,Month-to-month,True,Electronic check,0.304702,280.85,2.340417,-0.658758,-0.658758
3,True,True,True,False,-0.787401,True,False,Fiber optic,False,True,...,True,True,Month-to-month,True,Electronic check,1.104635,1237.85,3.173974,0.795726,0.795726
4,False,True,True,False,-1.194400,True,False,Fiber optic,False,False,...,True,False,Month-to-month,True,Mailed check,0.636624,267.40,2.971111,-0.173930,-0.173930
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7262,False,False,False,False,-0.787401,True,False,DSL,True,False,...,False,False,One year,False,Mailed check,-0.317653,742.90,1.904872,-0.173930,-0.173930
7263,True,False,True,False,-0.421101,True,True,Fiber optic,False,False,...,False,True,Month-to-month,True,Electronic check,0.676455,1873.70,2.838939,-0.173930,-0.173930
7264,True,False,False,False,-1.235100,True,False,DSL,False,True,...,False,False,Month-to-month,True,Mailed check,-0.478636,92.75,1.545833,-0.658758,-0.658758
7265,True,False,True,True,1.410394,True,False,DSL,True,False,...,False,True,Two year,False,Mailed check,0.103889,4627.65,2.302313,0.795726,0.795726


# **ONE HOT ENCONDING**

In [85]:
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder

In [94]:
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder
import pandas as pd

# Primeiro, defina as listas de colunas:

colunas_binarias = [
    'Genero', 'Idoso', 'Possui_Conjuge', 'Possui_Dependentes', 'Servico_Telefone',
    'Multiplas_Linhas', 'Seguranca_Online', 'Backup_Online', 'Protecao_Dispositivo',
    'Suporte_Tecnico', 'Streaming_TV', 'Streaming_Filmes', 'Fatura_Papel'  # parece ser booleana, mas vc mencionou fatura digital?
]

colunas_multiclasse = ['Tipo_Internet', 'Tipo_Contrato', 'Metodo_Pagamento']

# Variáveis explicativas: tudo menos Cancelou e ID_Cliente
variaveis_explicativas_x = df.drop(columns=['Cancelou', 'ID_Cliente'])

# Criar o transformador:
one_hot = make_column_transformer(
    (OneHotEncoder(drop='if_binary', sparse_output=False), colunas_binarias),
    (OneHotEncoder(drop='first', sparse_output=False), colunas_multiclasse),
    remainder='passthrough'
)

# Aplicar o fit_transform
variaveis_explicativas_x_encoded = one_hot.fit_transform(variaveis_explicativas_x)

# Obter os nomes das colunas novas
colunas_novas = one_hot.get_feature_names_out()

# Montar o dataframe final
df_tratados_x = pd.DataFrame(variaveis_explicativas_x_encoded, columns=colunas_novas)

# Opcional: resetar index para combinar com o df original
df_tratados_x.index = df.index

df_tratados_x


,onehotencoder-1__Genero_True,onehotencoder-1__Idoso_True,onehotencoder-1__Possui_Conjuge_True,onehotencoder-1__Possui_Dependentes_True,onehotencoder-1__Servico_Telefone_True,onehotencoder-1__Multiplas_Linhas_True,onehotencoder-1__Seguranca_Online_True,onehotencoder-1__Backup_Online_True,onehotencoder-1__Protecao_Dispositivo_True,onehotencoder-1__Suporte_Tecnico_True,...,onehotencoder-2__Tipo_Contrato_Two year,onehotencoder-2__Metodo_Pagamento_Credit card (automatic),onehotencoder-2__Metodo_Pagamento_Electronic check,onehotencoder-2__Metodo_Pagamento_Mailed check,remainder__Meses_de_Contrato,remainder__Valor_Mensal,remainder__Charges.Total,remainder__daily.account,remainder__Qtd_Servicos,remainder__Total_Servicos
0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,1.0,-0.950200,0.029206,593.30,2.197407,0.310898,0.310898
1,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,-0.950200,-0.159990,542.40,2.008889,-0.173930,-0.173930
2,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,-1.153700,0.304702,280.85,2.340417,-0.658758,-0.658758
3,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,1.0,0.0,-0.787401,1.104635,1237.85,3.173974,0.795726,0.795726
4,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,-1.194400,0.636624,267.40,2.971111,-0.173930,-0.173930
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7262,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,-0.787401,-0.317653,742.90,1.904872,-0.173930,-0.173930
7263,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,-0.421101,0.676455,1873.70,2.838939,-0.173930,-0.173930
7264,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,-1.235100,-0.478636,92.75,1.545833,-0.658758,-0.658758
7265,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,...,1.0,0.0,0.0,1.0,1.410394,0.103889,4627.65,2.302313,0.795726,0.795726


In [95]:
import pandas as pd
from statsmodels.stats.outliers_influence import variance_inflation_factor

# df_tratados_x: seu DataFrame já com encoding e normalização

# Para calcular VIF precisamos de uma matriz numpy
X = df_tratados_x.values

# Criar DataFrame para armazenar os VIFs
vif_data = pd.DataFrame()
vif_data["feature"] = df_tratados_x.columns

# Calcular VIF para cada variável
vif_data["VIF"] = [variance_inflation_factor(X, i) for i in range(X.shape[1])]

# Mostrar as variáveis com VIF alto (exemplo > 10)
print(vif_data.sort_values(by="VIF", ascending=False))

# Se quiser filtrar só as problemáticas:
print("\nVariáveis com VIF > 10:")
print(vif_data[vif_data["VIF"] > 10])


                                              feature         VIF
24                            remainder__Qtd_Servicos         inf
25                          remainder__Total_Servicos         inf
21                            remainder__Valor_Mensal  976.044086
13         onehotencoder-2__Tipo_Internet_Fiber optic  149.021181
23                           remainder__daily.account  110.438607
4              onehotencoder-1__Servico_Telefone_True  108.471770
14                  onehotencoder-2__Tipo_Internet_No  105.059607
5              onehotencoder-1__Multiplas_Linhas_True   40.855366
8          onehotencoder-1__Protecao_Dispositivo_True   32.974233
7                 onehotencoder-1__Backup_Online_True   32.938104
9               onehotencoder-1__Suporte_Tecnico_True   27.997451
6              onehotencoder-1__Seguranca_Online_True   27.587165
11             onehotencoder-1__Streaming_Filmes_True   21.729146
10                 onehotencoder-1__Streaming_TV_True   21.181270
22        

c:\Users\graci\OneDrive\Documentos\Challenge Telecom-2\.venv\Lib\site-packages\statsmodels\stats\outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


In [96]:
# Remove uma das colunas altamente correlacionadas
variaveis_explicativas_x = df.drop(columns=['Cancelou', 'ID_Cliente', 'Total_Servicos'])

one_hot = make_column_transformer(
    (OneHotEncoder(drop='if_binary', sparse_output=False), colunas_binarias),
    (OneHotEncoder(drop='first', sparse_output=False), colunas_multiclasse),
    remainder='passthrough'
)

variaveis_explicativas_x_encoded = one_hot.fit_transform(variaveis_explicativas_x)
colunas_novas = one_hot.get_feature_names_out()
df_tratados_x = pd.DataFrame(variaveis_explicativas_x_encoded, columns=colunas_novas)
df_tratados_x.index = df.index

# Calcular VIF novamente
X = df_tratados_x.values
vif_data = pd.DataFrame()
vif_data["feature"] = df_tratados_x.columns
vif_data["VIF"] = [variance_inflation_factor(X, i) for i in range(X.shape[1])]
print(vif_data.sort_values(by="VIF", ascending=False))


                                              feature         VIF
21                            remainder__Valor_Mensal  976.044086
24                            remainder__Qtd_Servicos  765.068331
13         onehotencoder-2__Tipo_Internet_Fiber optic  149.021181
23                           remainder__daily.account  110.438607
4              onehotencoder-1__Servico_Telefone_True  108.471770
14                  onehotencoder-2__Tipo_Internet_No  105.059607
5              onehotencoder-1__Multiplas_Linhas_True   40.855366
8          onehotencoder-1__Protecao_Dispositivo_True   32.974233
7                 onehotencoder-1__Backup_Online_True   32.938104
9               onehotencoder-1__Suporte_Tecnico_True   27.997451
6              onehotencoder-1__Seguranca_Online_True   27.587165
11             onehotencoder-1__Streaming_Filmes_True   21.729146
10                 onehotencoder-1__Streaming_TV_True   21.181270
22                           remainder__Charges.Total   10.752952
20        

# **Verificação da Proporção de Evasão**

In [87]:
# Contagem dos valores
contagem = df['Cancelou'].value_counts()

# Exibindo proporções
proporcao = df['Cancelou'].value_counts(normalize=True)

# Exibir os resultados
print("Contagem:\n", contagem)
print("\nProporção:\n", proporcao)


Contagem:
 Cancelou
False    5398
True     1869
Name: count, dtype: int64

Proporção:
 Cancelou
False    0.74281
True     0.25719
Name: proportion, dtype: float64


A verificação da proporção de evasão mostra que 74,3% dos clientes permanecem ativos e apenas 25,7% evadiram, o que evidencia um **desequilíbrio entre as classes**. Esse desbalanceamento pode afetar negativamente modelos preditivos, pois algoritmos de machine learning tendem a favorecer a classe majoritária. Como resultado, o modelo pode apresentar alta acurácia, mas baixa capacidade de identificar corretamente os clientes que evadem — justamente o grupo de maior interesse para ações estratégicas. Por isso, é necessário aplicar técnicas de balanceamento, como reamostragem ou uso de pesos nas classes, para garantir que o modelo aprenda a distinguir bem ambos os grupos e gere previsões mais confiáveis.

# **Balanceamento de Classes**


In [97]:
print(Y.value_counts(normalize=True))


Cancelou
False    0.74281
True     0.25719
Name: proportion, dtype: float64


In [103]:
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=42)
X_res, y_res = ros.fit_resample(df_tratados_x, Y)

print(pd.Series(y_res).value_counts())


ImportError: cannot import name '_deprecate_Xt_in_inverse_transform' from 'sklearn.utils.deprecation' (c:\Users\graci\OneDrive\Documentos\Challenge Telecom-2\.venv\Lib\site-packages\sklearn\utils\deprecation.py)